# 3. Motifs

In this tutorial, waveform motif identification will be demonstrated with simulated and ECoG data.

## Simulated Data

First, a timeseries containing two bursty, asymmetrical sine waves with a 1/f background will be simulated.

In [ ]:
from itertools import cycle
import numpy as np

from neurodsp.sim import sim_oscillation, sim_powerlaw
from neurodsp.plts import plot_time_series, plot_power_spectra
from neurodsp.spectral import compute_spectrum
from neurodsp.filt import filter_signal
from neurodsp.utils.norm import normalize_sig

from fooof import FOOOF

from ndspflow.motif import Motif
from ndspflow.optimize import refit

In [ ]:
# Settings
fs = 2000
n_seconds = 20
freq = 20

# Define signals
sig_lo_rdsym = sim_oscillation(1, fs, freq, cycle='asine', rdsym=.2, phase='min')
sig_hi_rdsym = sim_oscillation(1, fs, freq, cycle='asine', rdsym=.8, phase='min')

sig_lo_rdsym *= .8
sig_hi_rdsym *= .8

sig_ap = sim_powerlaw(n_seconds, fs, exponent=-2)

sig = np.zeros_like(sig_ap)

# Sample defintions
cyc_len = len(sig_lo_rdsym)
cyc_starts = np.arange(0, len(sig), cyc_len)
cyc_ends = np.arange(cyc_len, len(sig)+cyc_len, cyc_len)

# Alternate between bursts of .2 rdsym, .8 rdsym, and no burst
sig_zeros = np.zeros_like(sig_lo_rdsym)
choices = [sig_zeros, sig_lo_rdsym, sig_zeros, sig_hi_rdsym]
for start, end, ind in zip(cyc_starts, cyc_ends, cycle([0, 1, 2, 3])):
    sig[start:end] = np.sum((choices[ind] * 0.5,
                             sig_ap[start:end]), axis=0)

sig_pe = sig - sig_ap

In [ ]:
# Plot simulated signal
times = np.arange(0, len(sig)/fs, 1/fs)

xlim = (0, 20)
plot_time_series(times, sig, title='Combined Signal', xlim=xlim)
plot_time_series(times, sig_pe, title='Periodic Component', xlim=xlim)
plot_time_series(times, sig_ap, title='Aperiodic Component', xlim=xlim)

### Spectral Parametrization

Identifying motifs requires spectral parameterization to define center frequencies and bandwidths. Note, a list of tuples, defining center frequencies and bandwidth may be alternatively used.

In [ ]:
# Spectral parametrization
f_range = (1, 100)

freqs, powers = compute_spectrum(sig, fs, f_range=f_range)

fm = FOOOF()
fm.fit(freqs, powers, freq_range=f_range)

### Motif Extraction

Motif extraction involves using Bycyles to segement cycles by defining extrema and zero-crossings. Cycles within a peaks bandwidth are resampled to the center freqeuency, allowing a mean motif to be found. Each segmented waveform found within the bandwidth of a spectral peak is resample to the center frequency, allowing a mean waveform to be associated with each peak. Multiple waveforms at a given peak are distinguished using k-means clustering. Correlated cycles are extracted and a refined motif is recomputed.

In [ ]:
# Motif extraction
motif = Motif(
    corr_thresh=.25,
    min_clust_score=0.25,
    min_clusters=1,
    max_clusters=5,
    min_n_cycles=10
)

motif.fit(fm, sig, fs)
motif.plot(plot_fm_kwargs={'log_freqs': True})

Alternatively, emd may be used to identify modes with power above the aperiodic fit. The hilbert-huang transform of the sum of these modes provide frequency ranges to fit the periodic spectral component. Since the HHT is insensitive to harmonics, a single peak will be fit at 20hz. This method also helps prevent overfitting power spectra.

After refitting the spectra, motifs may be found from either the sum of modes or the original signal. Using the sum of modes helps limit the influence of the aperiodic component on motifs. This is especially useful for short duration oscillations.

In [ ]:
# Refit specparam
fm_refit, imf, pe_mask = refit(fm, sig, fs, f_range,
                               power_thresh=0.01, energy_thresh=.1)

# Recompute motifs
motif = Motif(corr_thresh=.1)
motif.fit(fm_refit, imf[pe_mask].sum(axis=0), fs, ttype='euclidean')
motif.plot(plot_fm_kwargs={'log_freqs': True})

In [ ]:
motif.sig = sig
motif.plot_spectra(0)

### Signal Decomposition

Once motifs are found, they may be transformed to each individual cycles using a variety of methods, including affine, euclidean and others. The resulting transformation matrix describes cycle-by-cycle variation, assuming the cycles are not stationary. Transformed motifs provide an estimate of the periodic signal. The difference between transformed motifs and cycle waveforms provides an estimate of the aperiodic signal. Spectra of the decomposed signals may be used to validate the aperiodic/periodic separation in the frequency domain.

In [ ]:
# Signal decomposition
motif.sig = sig
motif.decompose(ttype='euclidean')
motif.plot_decompose(0)

## LFP Data

Motif extraction and signal decomposition will be applied to a pubically available LFP from the CRCNS HC-2 dataset (Mizuseki K, Sirota A, Pastalkova E, Buzsáki G., Neuron. 2009 Oct 29;64(2):267-80. (http://www.ncbi.nlm.nih.gov/pubmed/19874793).

In [ ]:
# Normalize and filter the data
f_range = (1, 60)
sig_lfp = np.load('data/ca1.npy')
fs = 1250

sig_lfp = normalize_sig(sig_lfp, mean=0, variance=1)
sig_lfp = filter_signal(sig_lfp, fs, 'bandpass', f_range, remove_edges=False)

In [ ]:
# Spectral parameterization
freqs, powers = compute_spectrum(sig_lfp, fs, f_range=f_range)
fm = FOOOF(aperiodic_mode='knee')
fm.fit(freqs, powers, f_range)

In [ ]:
# Motif extraction
motif_lfp = Motif(corr_thresh=.01, max_clusters=1,  var_thresh=0.01)
motif_lfp.fit(fm, sig_lfp, fs)
motif_lfp.plot(plot_fm_kwargs={'log_freqs': True}) 

Next, the spectrum is refit using the EMD/HHT approach. The spectral fit doesn't change, indicating the intial fit was appropriate. However, the modes provide signal to aid in refining the motif, removing the influences of frequency outside of the peak range.

In [ ]:
# EMD based spectral refitting
fm_refit, imf, pe_mask = refit(fm, sig_lfp, fs, f_range,
                               power_thresh=0.1, energy_thresh=1)

# Motif extraction
motif_lfp = Motif(corr_thresh=0.1, max_clusters=1,  var_thresh=0.01)
motif_lfp.fit(fm_refit, imf[pe_mask].sum(axis=0), fs)
motif_lfp.plot(plot_fm_kwargs={'log_freqs': True}) 

In [ ]:
# Signal Decomposition
motif_lfp.sig = sig_lfp
motif_lfp.decompose()
motif_lfp.plot_decompose(0, xlim=(0, 10))

In [ ]:
motif_lfp.plot_spectra(0, f_range=f_range)

In [ ]:
# Plot the affine transformation parameters
motif_lfp.plot_transform(0, xlim=(0, 10))